In [7]:
!pip install -qU langchain

In [8]:
!pip install -U weaviate-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.0/433.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 29.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.73.0
    Uninstalling grpcio-1.73.0:
      Successfully uninstalled grpcio-1.73.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.

In [9]:
from IPython.display import display, Markdown

In [10]:
import weaviate
from weaviate.classes.init import Auth

In [11]:
from google.colab import userdata
weaviate_url = userdata.get("weviate_cluster")
weaviate_api_key = userdata.get("weviate_key")

In [12]:
client=weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

In [21]:
!pip install -qU sentence-transformers

In [22]:
!pip install -qU langchain-community pypdf

In [24]:
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cpu"}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs
)

In [26]:
from langchain_community.document_loaders import PyPDFLoader
pages=PyPDFLoader("/content/student_handbook.pdf",extract_images=True).load()

In [27]:
# pages

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
docs=text_splitter.split_documents(pages)

In [29]:
# docs

In [30]:
!pip install langchain-weaviate

In [31]:
from langchain_weaviate.vectorstores import WeaviateVectorStore
vector_db = WeaviateVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    client=client,
    by_text=False

    # docs, embeddings, client=client, by_text=False
)

In [32]:
print(vector_db.similarity_search('what is the registration process in iit bhilai?',k=3)[0].page_content)

parts of this process rests with the students. If a student fails to complete the registration 
process within the specified time, he/she shall be deemed to be on una uthorized absence 
from the Institute and suitable disciplinary action including the termination of the studentship 
shall be carried out by the Institute. 
Academic Pre-registration 
• All students must pre -register for the courses scheduled to be taken in the next 
semester as per schedule specified in the Academic Calendar. 
• It will be the responsibility of the students to ensure that they meet the eligibility criteria, 
pre-requisite criteria if any. Students may only register for the number of courses as 
permitted by semester load rules and must also ensure that there are no timetable 
clashes amongst the courses. The students shall need to submit the fully filled 
registration form  along with the permissions from the instructors if needed  for the 
approval of the DPGC/DUGC.


In [33]:
print(vector_db.similarity_search('what is the registration process in iit bhilai?',k=3)[1].page_content)

registration must obtain prior permission from Dean of Academic Affairs. They shall be 
treated as non-reporting students if they fail to do so. 
Administrative Registration 
• All students must pay for all dues, including the hostel dues, academic fees, any other 
dues that the Institute may prescribe from time to time etc. during the administrative 
registration. 
• All students whose ID card expires must get it renewed as pe r the norms of the 
Institute. 
• All students must carry out registration of facilities such as various labs, library etc. as 
specified by the Institute during the administrative registration. 
• Any failure to do the administrative registration as per the stipu lated time table shall 
attract penalty, disciplinary action and/or termination of the program. 
• A doctorate student who has submitted the thesis and is waiting for defense shall 
register for zero credits if he/she wishes to stay on campus. The student shall pay all


In [34]:
from re import template
from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say this is not mentioned in the student-handbook.
and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [35]:
prompt=ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say this is not mentioned in the student-handbook.\nand keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [36]:
!pip install -qU langchain-huggingface

In [37]:
from langchain_huggingface import HuggingFaceEndpoint
huggingface_token=userdata.get("h_key")

In [38]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    huggingfacehub_api_token=huggingface_token,
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03
)

chat_model = ChatHuggingFace(llm=llm)

In [39]:
chat_model.invoke("what is the registration process in iit bhilai?").content

"The registration process for IIT Bhilai is as follows:\n1. Visit the official website of IIT Bhilai.\n2. Click on the 'Admissions' tab.\n3. Click on the 'Undergraduate Admissions' tab.\n4. Click on the 'Registration' link.\n5. Fill in the necessary details such as name, email, phone number, and password.\n6. Submit the registration form.\n7. You will receive a registration confirmation on your registered email ID.\n8. After registration, you can apply for various undergraduate programs offered by IIT Bhilai.\n"

In [40]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

output_parser=StrOutputParser()

In [41]:
retriever=vector_db.as_retriever()

In [42]:
retriever.get_relevant_documents("what is the registration process in iit bhilai?")

/tmp/ipython-input-42-651517415.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("what is the registration process in iit bhilai?")


[Document(metadata={'author': 'IIT BHILAI', 'moddate': datetime.datetime(2023, 6, 30, 19, 57, 14, tzinfo=datetime.timezone(datetime.timedelta(seconds=19800))), 'page': 10.0, 'page_label': '11', 'total_pages': 28.0, 'creationdate': datetime.datetime(2023, 6, 30, 19, 57, 14, tzinfo=datetime.timezone(datetime.timedelta(seconds=19800))), 'source': '/content/student_handbook.pdf', 'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365'}, page_content='parts of this process rests with the students. If a student fails to complete the registration \nprocess within the specified time, he/she shall be deemed to be on una uthorized absence \nfrom the Institute and suitable disciplinary action including the termination of the studentship \nshall be carried out by the Institute. \nAcademic Pre-registration \n• All students must pre -register for the courses scheduled to be taken in the next \nsemester as per schedule specified in the Academic Calendar. \n• It

In [43]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | chat_model
    | output_parser
)

In [44]:
res=rag_chain.invoke("tell me the grading system of iit bhilai")

In [45]:
display(Markdown(res))

The grading system of IIT Bhilai for regular courses is as follows:
- A+: 10 points
- A: 10 points
- A–: 9 points
- B+: 8 points
- B: 7 points
- B–: 6 points
- C+: 5 points
- C: 4 points
- D: 0 points
- F: 0 points
- FS: 0 points
- I: 0 points

Grades for Thesis and Candidacy examinations are as follows:
- A: 10 points
- B: 8 points
- C: 6 points
- F: 0 points
- I: 0 points

[It's important to note that all the information provided is based on the retrieved context and might change over time.]


In [46]:
res=rag_chain.invoke("what is the registration process in iit bhilai?")

In [47]:
display(Markdown(res))

Students first need to pre-register for the courses they are scheduled to take in the next semester. They should ensure they meet the eligibility criteria and pre-requisite criteria. If they need instructor permissions, they must submit a fully filled registration form along with the necessary permissions. If any student fails to complete the registration process within the specified time, they may face disciplinary action.


In [48]:
res=rag_chain.invoke("This student notebook is Applicable for students admitted till?")

In [50]:
display(Markdown(res))

2022-23-W semester
